<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=320b51bcc887502b3bfb1dd46b2df31dfbf5803d66d2b108090727a4f1c1f350
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-27 12:22:50
-------------------
qualified stocks: 89
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.00 K
Current:  1.45 C
-------------------
Today PnL: 64.14 K (0.44%)
Current PnL: -19.11 L (-12.53%)
CY Booked + Current PnL: -7.01 L (-4.6%)
-------------------
Total profit:  2.48 L
Total loss:  -21.59 L
-------------------
Total Booked + Current PnL: 19.97 L (15.89%)
Total Booked PnL: 39.08 L (31.11%)
Curr Year Booked PnL: 12.10 L (8.34%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.01 L (59.98%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.59%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-7.66,72.0,X-LC,3.50,231223.0,20857.0,7931.0,2.11,9.91,3.43,13.68,37.0,2.63,1.59,28.08,XY25,NTT,REFINERIES
86,VOLTAS,1530.0,3.12,60.0,X-MC,2.80,216210.0,24468.0,13297.0,0.71,12.76,6.15,19.69,99.0,1.84,1.49,20.43,XY25,NTT,AC
42,ITC,452.0,-35.81,69.0,X-LC,1.68,206651.0,6513.0,14383.0,1.39,3.25,6.96,10.44,4.0,0.45,1.42,9.00,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-9.44,42.0,X-MC,3.20,89505.0,-1173.0,16084.0,-0.32,-1.29,17.97,16.44,101.0,-0.07,0.62,18.62,X40,ATH,PHARMA
79,TTKPRESTIG,770.0,96.16,49.0,M-SC,2.07,84495.0,-16282.0,16375.0,0.02,-16.16,19.38,0.09,245.0,-0.99,0.58,11.01,OX40N,NTT,DURABLES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SATIN,274.00,-22.47,72.0,H-SC,4.15,239571.0,-36713.0,179918.0,6.16,-13.29,75.10,51.83,126.0,-0.20,1.65,16.71,XY24,NTT,FINANCE
37,INDIAMART,4810.62,-53.33,56.0,H-SC,7.02,130210.0,6874.0,124754.0,3.63,5.57,95.81,106.72,119.0,0.06,0.90,30.25,AR,ATH,MISC
12,BANKINDIA,190.00,-18.01,73.0,H-MC,15.08,212259.0,32451.0,79958.0,3.07,18.05,37.67,62.52,88.0,0.41,1.46,58.29,XR,NTT,BANKS
80,UJJIVANSFB,60.00,128.70,72.0,H-SC,16.27,144369.0,1890.0,17627.0,2.79,1.33,12.21,13.70,163.0,0.11,1.00,70.40,OX40N,NTT,BANKS
15,BSOFT,836.99,-11.31,67.0,H-SC,14.32,99654.0,-40999.0,113775.0,2.45,-29.15,114.17,51.74,131.0,-0.36,0.69,15.30,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SHALBY,327.0,1128.07,46.0,M-SC,22.05,167048.0,-14095.0,59569.0,-1.77,-7.78,35.66,25.10,235.0,-0.24,1.15,33.87,XY24,NTT,HEALTHCARE
20,COFFEEDAY,80.0,-45.67,49.0,L-SC,27.42,81504.0,-32045.0,72098.0,-1.71,-28.22,88.46,35.27,268.0,-0.44,0.56,99.48,XR,NTT,HOTELS
38,INDIGOPNTS,1408.0,92.52,34.0,M-SC,4.88,126232.0,-48327.0,48359.0,-1.41,-27.68,38.31,0.02,221.0,-1.00,0.87,9.80,OX40N,NTT,PAINTS
83,VAIBHAVGBL,521.0,53.17,57.0,H-SC,4.52,132822.0,-49953.0,161020.0,-1.35,-27.33,121.23,60.77,125.0,-0.31,0.92,18.96,XR,NTT,JEWELLERY
84,VALIANTORG,838.0,-286.53,40.0,H-SC,8.37,103700.0,-59905.0,181216.0,-1.29,-36.62,174.75,74.15,139.0,-0.33,0.71,32.93,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.0,41.94,48.0,M-MC,9.71,223826.0,-1136.0,168787.0,0.40,-0.51,75.41,74.53,192.0,-0.01,1.54,31.59,XY24,BTT,STEEL
36,IEX,219.0,-29.93,63.0,H-SC,17.18,200402.0,448.0,96774.0,0.43,0.22,48.29,48.63,115.0,0.00,1.38,11.80,XR,NTT,MISC
57,RECLTD,446.0,45.36,48.0,M-LC,5.70,203225.0,55.0,42068.0,0.41,0.03,20.70,20.74,55.0,0.00,1.40,6.89,XY25,NTT,FINANCE
88,WIPRO,420.0,-13.79,48.0,M-LC,5.90,151336.0,391.0,109068.0,0.46,0.26,72.07,72.51,53.0,0.00,1.04,6.30,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.0,128.70,72.0,H-SC,16.27,144369.0,1890.0,17627.0,2.79,1.33,12.21,13.70,163.0,0.11,1.00,70.40,OX40N,NTT,BANKS
46,KANSAINER,340.0,-67.06,57.0,H-SC,2.54,227718.0,-41949.0,78289.0,-0.37,-15.56,34.38,13.47,138.0,-0.54,1.57,15.79,XY24,NTT,PAINTS
79,TTKPRESTIG,770.0,96.16,49.0,M-SC,2.07,84495.0,-16282.0,16375.0,0.02,-16.16,19.38,0.09,245.0,-0.99,0.58,11.01,OX40N,NTT,DURABLES
74,TATAELXSI,9161.0,-21.19,59.0,H-MC,7.29,106524.0,-25311.0,67536.0,1.13,-19.20,63.40,32.03,98.0,-0.37,0.73,19.32,OX40N,NTT,IT
68,SIS,528.0,2064.25,51.0,H-SC,3.18,87338.0,-23694.0,47302.0,-0.26,-21.34,54.16,21.26,156.0,-0.50,0.60,18.00,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.00,128.70,72.0,H-SC,16.27,144369.0,1890.0,17627.0,2.79,1.33,12.21,13.70,163.0,0.11,1.00,70.40,OX40N,NTT,BANKS
57,RECLTD,446.00,45.36,48.0,M-LC,5.70,203225.0,55.0,42068.0,0.41,0.03,20.70,20.74,55.0,0.00,1.40,6.89,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.16,52.0,M-LC,10.95,210866.0,5790.0,106783.0,-0.40,2.82,50.64,54.89,52.0,0.05,1.45,27.60,X5K,ATH,METALS
47,KPIGREEN,731.05,8.25,67.0,H-SC,5.22,129049.0,3752.0,55181.0,-1.08,2.99,42.76,47.03,141.0,0.07,0.89,61.58,MH,ATH,POWER
81,UNIONBANK,163.00,-10.08,61.0,M-LC,9.27,162943.0,22103.0,22388.0,1.09,15.69,13.74,31.59,66.0,0.99,1.12,46.34,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-18.01,73.0,H-MC,15.08,212259.0,32451.0,79958.0,3.07,18.05,37.67,62.52,88.0,0.41,1.46,58.29,XR,NTT,BANKS
87,WHIRLPOOL,2270.00,-38.52,64.0,M-SC,3.56,127411.0,12613.0,81428.0,1.04,10.99,63.91,81.92,223.0,0.15,0.88,50.83,XR,NTT,DURABLES
80,UJJIVANSFB,60.00,128.70,72.0,H-SC,16.27,144369.0,1890.0,17627.0,2.79,1.33,12.21,13.70,163.0,0.11,1.00,70.40,OX40N,NTT,BANKS
47,KPIGREEN,731.05,8.25,67.0,H-SC,5.22,129049.0,3752.0,55181.0,-1.08,2.99,42.76,47.03,141.0,0.07,0.89,61.58,MH,ATH,POWER
37,INDIAMART,4810.62,-53.33,56.0,H-SC,7.02,130210.0,6874.0,124754.0,3.63,5.57,95.81,106.72,119.0,0.06,0.90,30.25,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-53.83,16.0,X-LC,15.22,146398.0,-130125.0,240195.0,-0.63,-47.06,164.07,39.81,54.0,-0.54,1.01,3.19,XY24,NTT,AUTO
45,JSWINFRA,342.00,-20.48,36.0,X-MC,5.41,199395.0,-952.0,31465.0,0.60,-0.48,15.78,15.23,178.0,-0.03,1.37,31.62,X40N,NTT,REALTY
24,DMART,5391.45,-16.66,37.0,X-LC,10.10,101926.0,-660.0,27469.0,0.78,-0.64,26.95,26.13,38.0,-0.02,0.70,24.76,X40N,ATH,FMCG
52,PAGEIND,51769.93,-27.18,41.0,X-MC,6.89,82230.0,-1280.0,21306.0,0.22,-1.53,25.91,23.98,82.0,-0.06,0.57,4.20,X40,ATH,APPARELS
54,QUESS,986.00,-47.66,41.0,X-SC,38.67,52242.0,-12764.0,163690.0,1.90,-19.63,313.33,232.18,198.0,-0.08,0.36,1.90,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-1.76,51.0,H-LC,0.91,158050.0,-28045.0,75532.0,0.28,-15.07,47.79,25.51,15.0,-0.37,1.09,16.47,AR,ATH,ELECTRICAL
42,ITC,452.00,-35.81,69.0,X-LC,1.68,206651.0,6513.0,14383.0,1.39,3.25,6.96,10.44,4.0,0.45,1.42,9.00,X40,NTT,FMCG
86,VOLTAS,1530.00,3.12,60.0,X-MC,2.80,216210.0,24468.0,13297.0,0.71,12.76,6.15,19.69,99.0,1.84,1.49,20.43,XY25,NTT,AC
53,PGHH,17907.65,-29.75,41.0,X-MC,3.06,203145.0,2325.0,65474.0,-0.07,1.16,32.23,33.76,80.0,0.04,1.40,6.42,X40,ATH,FMCG
29,HAVELLS,2069.16,-6.46,44.0,X-MC,3.08,312207.0,-15784.0,122323.0,-0.55,-4.81,39.18,32.48,92.0,-0.13,2.15,6.06,X40,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-47.66,41.0,X-SC,38.67,52242.0,-12764.0,163690.0,1.90,-19.63,313.33,232.18,198.0,-0.08,0.36,1.90,XY24,NTT,MISC
21,COLPAL,3767.14,-4.98,43.0,X-MC,8.14,221610.0,-41755.0,155105.0,-1.04,-15.85,69.99,43.04,84.0,-0.27,1.53,2.88,XY25,ATH,FMCG
75,TATAMOTORS,1065.00,-53.83,16.0,X-LC,15.22,146398.0,-130125.0,240195.0,-0.63,-47.06,164.07,39.81,54.0,-0.54,1.01,3.19,XY24,NTT,AUTO
3,ACC,3906.00,-38.07,47.0,X-MC,2.84,184230.0,-53721.0,206374.0,-0.42,-22.58,112.02,64.15,174.0,-0.26,1.27,3.34,XY24,BTT,CEMENT
10,BAJAJHFL,181.50,-14.48,46.0,X-MC,5.80,180318.0,-20188.0,117351.0,-0.24,-10.07,65.08,48.45,90.0,-0.17,1.24,3.98,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-47.66,41.0,X-SC,38.67,52242.0,-12764.0,163690.0,1.90,-19.63,313.33,232.18,198.0,-0.08,0.36,1.90,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.18,41.0,X-MC,6.89,82230.0,-1280.0,21306.0,0.22,-1.53,25.91,23.98,82.0,-0.06,0.57,4.20,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.30,45.0,X-SC,4.66,82720.0,-62440.0,138366.0,-0.95,-43.01,167.27,52.31,136.0,-0.45,0.57,10.73,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.44,42.0,X-MC,3.20,89505.0,-1173.0,16084.0,-0.32,-1.29,17.97,16.44,101.0,-0.07,0.62,18.62,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.15,47.0,X-SC,4.35,96878.0,-31792.0,77096.0,0.18,-24.71,79.58,35.20,219.0,-0.41,0.67,11.19,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.08,59.0,X-LC,12.04,290375.0,-55581.0,122277.0,0.85,-16.07,42.11,19.28,1.0,-0.45,2.00,7.35,X40,ATH,IT
40,INFY,2275.00,-17.80,59.0,X-LC,6.91,320544.0,7656.0,164022.0,0.17,2.45,51.17,54.87,3.0,0.05,2.21,10.93,X40,BTT,IT
42,ITC,452.00,-35.81,69.0,X-LC,1.68,206651.0,6513.0,14383.0,1.39,3.25,6.96,10.44,4.0,0.45,1.42,9.00,X40,NTT,FMCG
85,VBL,671.64,-17.33,56.0,X-LC,9.73,297468.0,-18374.0,131035.0,1.05,-5.82,44.05,35.67,5.0,-0.14,2.05,7.23,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,50.0,H-LC,6.17,249960.0,-11659.0,130879.0,0.51,-4.46,52.36,45.57,7.0,-0.09,1.72,6.36,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6972.22,58.0,L-SC,7.28,77613.0,-16197.0,92817.0,-1.08,-17.27,119.59,81.67,269.0,-0.17,0.54,50.30,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,27.42,81504.0,-32045.0,72098.0,-1.71,-28.22,88.46,35.27,268.0,-0.44,0.56,99.48,XR,NTT,HOTELS
50,MASFIN,398.61,-19.15,50.0,H-SC,9.20,91890.0,-6090.0,27696.0,-0.62,-6.22,30.14,22.05,152.0,-0.22,0.63,33.57,XR,ATH,FINANCE
87,WHIRLPOOL,2270.00,-38.52,64.0,M-SC,3.56,127411.0,12613.0,81428.0,1.04,10.99,63.91,81.92,223.0,0.15,0.88,50.83,XR,NTT,DURABLES
47,KPIGREEN,731.05,8.25,67.0,H-SC,5.22,129049.0,3752.0,55181.0,-1.08,2.99,42.76,47.03,141.0,0.07,0.89,61.58,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-197.73,79.0,M-SC,26.54,168030.0,18810.0,125367.0,-0.82,12.61,74.61,96.62,208.0,0.15,1.16,81.84,XY25,NTT,FINANCE
5,ANGELONE,3033.00,18.37,74.0,X-SC,5.61,209494.0,18488.0,39217.0,1.60,9.68,18.72,30.21,157.0,0.47,1.44,32.58,X40N,NTT,FINANCE
69,SONACOMS,806.63,-32.06,71.0,M-SC,8.65,84569.0,-16590.0,56594.0,0.95,-16.40,66.92,39.54,202.0,-0.29,0.58,19.25,AR,ATH,AUTO
60,REPCOHOME,880.00,-55.29,70.0,H-SC,3.46,256655.0,-29850.0,287197.0,-0.65,-10.42,111.90,89.82,134.0,-0.10,1.77,35.79,XY24,NTT,FINANCE
71,SURYODAY,240.00,73.25,66.0,H-SC,13.16,158967.0,-20104.0,85826.0,1.39,-11.23,53.99,36.71,135.0,-0.23,1.10,57.73,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.38
1,25,43.43
2,50,74.11


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.78
LC    31.98
MC    24.18
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.66
X40      16.19
XY25     11.99
XR       11.27
X40N     10.49
AR        7.75
OX40N     7.44
X200      1.74
X5K       1.45
SR        1.07
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.23
X-LC    21.41
X-MC    18.54
M-SC    12.56
M-LC     5.01
X-SC     4.56
H-LC     4.55
H-MC     3.80
M-MC     1.54
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.35,-3.49,37.26
IT,12.45,-16.33,78.72
FINANCE,12.02,-8.38,57.51
MISC,6.82,-16.14,80.50
BANKS,6.58,-4.63,59.67
ELECTRICAL,5.72,-8.73,47.28
PAINTS,5.48,-16.06,33.37
INSURANCE,3.78,-0.73,38.29
AUTO,2.74,-44.40,104.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3269403.0,22
XR,1314427.0,14
AR,1166050.0,9
X40,807373.0,12
X40N,634612.0,9
OX40N,566678.0,10
XY25,443264.0,8
SR,266243.0,2
X5K,106783.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3337120.0,25
M-SC,1221890.0,15
X-MC,1187753.0,14
X-LC,1031055.0,13
X-SC,517677.0,6
H-MC,327540.0,3
M-LC,280307.0,4
H-LC,277520.0,3
L-SC,254657.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1147614.0      6
           AR         836213.0      5
           XR         772030.0      7
M-SC       XY24       694262.0      6
X-MC       XY24       583052.0      4
X-LC       X40        442794.0      5
           XY24       309564.0      2
X-MC       X40        287483.0      6
X-SC       X40N       276891.0      4
H-SC       SR         266243.0      2
           OX40N      259839.0      4
X-LC       X40N       208905.0      3
H-LC       AR         206411.0      2
H-MC       XY24       180046.0      1
M-MC       XY24       168787.0      1
X-MC       XY25       168402.0      2
L-SC       XR         164915.0      2
X-SC       XY24       163690.0      1
M-SC       OX40N      149561.0      4
X-MC       X40N       148816.0      2
M-SC       XR         129274.0      2
           XY25       125367.0      1
           AR         123426.0      2
M-LC       XR         109068.0      1
           X5K        106783.0      1
L-SC       OX40N       89742.0      1
H-MC       XR          79958.0      1
X-SC       X40         77096.0      1
H-LC       X200        71109.0      1
X-LC       XY25        69792.0      3
H-MC       OX40N       67536.0      1
L-MC       XR          59182.0      1
H-SC       MH          55181.0      1
M-LC       XY25        42068.0      1
L-LC       XY25        37635.0      1
M-LC       XY24        22388.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
